# DeepHit: A Deep Learning Approach to Survival Analysis with Competing Risks

In [5]:
import kagglehub

from src.model import DeepHit
from src.tools import tools_test

/Users/hedmana/Projects/personal/deephit-survival-analysis/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
print(DeepHit())
print(tools_test())

DeepHit Model for Survival Analysis with Competing Risks
Tools are working properly.


In [7]:
# Download data from kaggle
path = kagglehub.dataset_download("raghadalharbi/breast-cancer-gene-expression-profiles-metabric")

print("Path to dataset files:", path)

100%|██████████| 2.72M/2.72M [00:00<00:00, 4.10MB/s]

Extracting files...
Path to dataset files: /Users/hedmana/.cache/kagglehub/datasets/raghadalharbi/breast-cancer-gene-expression-profiles-metabric/versions/1
